# Karabük 2025 Orman Yangınları: Etki Analizi ve İzleme Raporu

**Hazırlayan:** [Kullanıcı Adı/Kurum]
**Tarih:** 07 Aralık 2025

---

## 1. Yönetici Özeti
Bu rapor, 2025 yaz sezonunda Karabük ilinde meydana gelen orman yangınlarının çevresel etkilerini, yanan alanların büyüklüğünü ve hasar derecesini bilimsel yöntemlerle ortaya koymaktadır. Analiz, **Sentinel-2** uydu görüntüleri ve **Google Earth Engine (GEE)** platformu kullanılarak gerçekleştirilmiştir.

### Öne Çıkan Bulgular
*   **Hassas Maskeleme:** Tarım arazileri ve yerleşim yerleri **ESA WorldCover 2021** verisi kullanılarak analizden çıkarılmış, sadece orman ve çalılık alanlara odaklanılmıştır. Bu sayede hasat edilmiş tarlaların "yanmış" gibi görünmesi engellenmiştir.
*   **Gelişmiş Metrikler:** Standart dNBR yerine, bitki yoğunluğu farklarına daha duyarlı olan **RBR (Relativized Burn Ratio)** indeksi kullanılmıştır.
*   **Görsel Doğrulama:** Sonuçlar, "False Color" (Kızılötesi vurgulu) ve "Transparent Overlay" (Şeffaf katman) haritaları ile görsel olarak doğrulanmıştır.

## 2. Metodoloji

Çalışmada kullanılan yöntemler uluslararası uzaktan algılama standartlarına (USGS, UN-SPIDER) uygundur.

### 2.1. Veri Kaynakları
*   **Uydu:** Sentinel-2 (MSI Sensörü) - 10m/20m Çözünürlük
*   **Referans (Öncesi) Dönem:** 01 Temmuz - 20 Temmuz 2025 (Yangın öncesi en sağlıklı bitki örtüsü)
*   **Analiz (Sonrası) Dönem:** 05 Eylül - 30 Eylül 2025 (Yangın ve duman etkisinin bittiği dönem)

### 2.2. Kullanılan İndeksler
Yangın şiddetini belirlemek için aşağıdaki spektral indeksler kullanılmıştır:

1.  **NBR (Normalized Burn Ratio):** `(NIR - SWIR) / (NIR + SWIR)`
    *   Sağlıklı bitki örtüsü ve yanmış alan ayrımı için temel indekstir.
2.  **dNDVI (Delta NDVI):** Vejetasyon kaybını doğrulamak için yardımcı indeks.
3.  **RBR (Relativized Burn Ratio):** `dNBR / (Pre_NBR + 1.001)`
    *   **Neden RBR?** dNBR, yoğun ormanlarda ve seyrek çalılıklarda farklı sonuçlar verebilir. RBR, yangın öncesi bitki yoğunluğunu hesaba katarak daha normalize ve doğru bir hasar derecesi sunar.

### 2.3. Sınıflandırma ve Maskeleme
*   **Arazi Örtüsü Maskesi:** **ESA WorldCover v200** verisi kullanılarak sadece **"Tree Cover" (Sınıf 10)** ve **"Shrubland" (Sınıf 20)** pikselleri analize dahil edilmiştir. **Tarım arazileri (Cropland) tamamen maskelenmiştir.**
*   **Şiddet Sınıfları (RBR Eşikleri):**
    *   🟢 **Yanmamış / Çok Düşük:** < 0.10
    *   🟡 **Düşük Şiddet:** 0.10 - 0.35
    *   🟠 **Orta Şiddet:** 0.35 - 0.75
    *   🔴 **Yüksek Şiddet:** > 0.75

In [1]:
# --- KURULUM ve AYARLAR ---
import os
import json
import ee
import pandas as pd
from IPython.display import IFrame, display, HTML, Image

# Proje Modülleri
from gee.pipeline import run_pipeline
from gee.utils import ee_init
from gee.aoi import get_aoi

# # Görünüm Ayarları
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', 100)
# display(HTML("<style>.container { width:100% !important; }</style>"))

# print("🔑 GEREKLİ EYLEM: Açılan pencerede linke tıklayın, yetki verin ve kodu buraya yapıştırın.")
# print("‼️ DİKKAT: 'Project to use' sorulduğunda 'karabuk-orman-yangini-2025' projesini seçin.")

# ee.Authenticate(force=True)

MY_PROJECT = 'karabuk-orman-yangini-2025'

try:
    ee_init(project=MY_PROJECT)
    print(f"✅ Google Earth Engine bağlantısı başarılı. Proje: {MY_PROJECT}")
except Exception as e:
    print(f"❌ Bağlantı Hatası: {e}")

✅ Google Earth Engine bağlantısı başarılı. Proje: karabuk-orman-yangini-2025


In [2]:
# --- PARAMETRELER ---
pre_start = '2025-07-01'
pre_end   = '2025-07-20'
post_start = '2025-09-05'
post_end   = '2025-09-30'

base_out_dir = 'results'

def show_map(path, title="Harita", height=600):
    """Haritayı IFrame içinde gösterir."""
    if os.path.exists(path):
        display(HTML(f"<h3 style='color:#333'>{title}</h3>"))
        display(IFrame(src=path, width='100%', height=height))
    else:
        print(f"⚠️ Dosya bulunamadı: {path}")

print(f"📅 Analiz Dönemi: {pre_start}___{pre_end} (Öncesi) <---> {post_start}___{post_end} (Sonrası)")

📅 Analiz Dönemi: 2025-07-01___2025-07-20 (Öncesi) <---> 2025-09-05___2025-09-30 (Sonrası)


## 3. Bölüm 1: İl Geneli Tarama (Overview)
Bu aşamada tüm il sınırları taranarak büyük yangın kümeleri tespit edilir. **RBR** haritası üzerinden genel durum gözlemlenir.

In [4]:
print("🔄 İl Geneli Analiz Başlatılıyor...")

province_out_dir = os.path.join(base_out_dir, "il_geneli")
province_bbox_path = "karabuk_province_bbox.geojson"

try:
    # İl sınırlarını al
    full_province_geom = get_aoi("KARABUK_PROVINCE")
    
    # Geçici bbox dosyası oluştur
    with open(province_bbox_path, "w", encoding="utf-8") as f:
        json.dump({"type": "Feature", "properties": {"name": "BBOX"}, "geometry": full_province_geom.bounds().getInfo()}, f)

    # PİPELİNE ÇALIŞTIR
    province_outputs = run_pipeline(
        pre_start=pre_start, pre_end=pre_end,
        post_start=post_start, post_end=post_end,
        aoi_geojson=province_bbox_path,
        out_dir=province_out_dir,
        area_scale=200,     # Geniş alan için performans optimizasyonu
        min_patch_ha=5,     # 5 hektardan küçük lekeleri yoksay
        skip_severity=False,
        overlay_boundary=full_province_geom
    )
    print("✅ İl analizi tamamlandı.")
    
    # Özet İstatistikleri Göster
    if 'summary_csv' in province_outputs:
        df = pd.read_csv(province_outputs['summary_csv'])
        display(df)
        
except Exception as e:
    print(f"❌ HATA: {e}")
finally:
    if os.path.exists(province_bbox_path): os.remove(province_bbox_path)

🔄 İl Geneli Analiz Başlatılıyor...
✅ İl analizi tamamlandı.


### 3.1. İl Çıktıları
Aşağıda il genelindeki RBR (Hasar) haritası ve False Color görüntüleri yer almaktadır.

In [5]:
if 'province_outputs' in locals():
    # False Color Map: Yangınları Vurgular
    show_map(province_outputs.get('falsecolor_map'), "🔥 False Color Haritası (Kızılötesi Vurgulu)")
    
    # RBR Severity Map
    show_map(province_outputs.get('severity_map'), "🌍 RBR Hasar Şiddeti Haritası (ESA Maskeli)")

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

## 4. Bölüm 2: Kritik Bölgelerin Detaylı Analizi
Yangınların yoğunlaştığı 7 kritik bölge için **yüksek çözünürlüklü (10m)** analizler aşağıda sunulmuştur. Her bölge için:
1.  **Severity Map:** Hasar derecesi.
2.  **Transparent Overlay:** Gerçek görüntü üzerine şeffaf hasar katmanı (Yer doğrulaması için).
3.  **İstatistikler:** Etkilenen orman alanı (hektar).

In [3]:
FIRE_ZONES = [
    {"name": "1_Aladag",       "center": [32.55, 41.13]},  # Safranbolu Güneyi
    {"name": "2_Cumayani",     "center": [32.70, 41.20]},  # Merkez Güney
    {"name": "3_Buyuk_Ovacik", "center": [32.80, 41.25]},  # En büyük yangın
    {"name": "4_Kisla",        "center": [32.85, 41.28]},  # Ovacık Doğusu
    {"name": "5_Soguksu",      "center": [32.60, 41.20]},  # Merkez - Soğuksu TOKİ (Düzeltildi: 37.00E -> 32.60E)
    {"name": "6_Toprakcuma",   "center": [33.10, 41.30]},  # Kuzeybatı Sınır
    {"name": "7_Eflani",       "center": [33.20, 41.35]}   # Kuzey - Eflani
]

zone_results = {}
print("🔄 Bölgesel analiz döngüsü başlatılıyor (7 Kritik Bölge)...")

for zone in FIRE_ZONES:
    name = zone['name']
    print(f"\n📍 Analiz Ediliyor: {name}...")
    
    zone_dir = os.path.join(base_out_dir, "yanginlar", name)
    center = ee.Geometry.Point(zone['center'])
    # Analiz alanı: Merkezden 6km yarıçap (12x12km kutu)
    zone_geom = center.buffer(6000).bounds()
    
    temp_geojson = f"temp_{name}.geojson"
    with open(temp_geojson, "w") as f:
        json.dump({"type": "Feature", "geometry": zone_geom.getInfo()}, f)
        
    try:
        # Pipeline Çalıştır
        outputs = run_pipeline(
            pre_start='2025-06-15', pre_end='2025-07-20',   # Esnek tarih aralığı
            post_start='2025-09-05', post_end='2025-10-15',
            aoi_geojson=temp_geojson,
            out_dir=zone_dir,
            area_scale=10,       # 10m Çözünürlük (Kritik bölgeler için)
            min_patch_ha=0.5,    # Küçük detaylar
            skip_severity=False
        )
        zone_results[name] = outputs

        
        
    except Exception as e:
        print(f"⚠️ {name} analizinde hata: {e}")
    finally:
        if os.path.exists(temp_geojson): os.remove(temp_geojson)

print("✅ Tüm kritik bölgeler tamamlandı.")

🔄 Bölgesel analiz döngüsü başlatılıyor (7 Kritik Bölge)...

📍 Analiz Ediliyor: 1_Aladag...

📍 Analiz Ediliyor: 2_Cumayani...

📍 Analiz Ediliyor: 3_Buyuk_Ovacik...

📍 Analiz Ediliyor: 4_Kisla...

📍 Analiz Ediliyor: 5_Soguksu...

📍 Analiz Ediliyor: 6_Toprakcuma...

📍 Analiz Ediliyor: 7_Eflani...
✅ Tüm kritik bölgeler tamamlandı.


## 5. Sonuçlar ve Tartışma

### 5.1. Bölgesel Hasar İstatistikleri
Aşağıdaki tablo, her bir bölge için tespit edilen **Yüksek (Kırmızı)** ve **Orta (Turuncu)** şiddetli yanık alanlarını göstermektedir.

In [ ]:
summary_list = []

for z_name, res in zone_results.items():
    if 'summary_csv' in res:
        d = pd.read_csv(res['summary_csv'])
        metrics = dict(zip(d['Key'], d['Value']))
        summary_list.append({
            "Bölge": z_name,
            "Yüksek Şiddet (Ha)": float(metrics.get('alan_ha_sinif_4', 0)),
            "Orta Şiddet (Ha)": float(metrics.get('alan_ha_sinif_3', 0)),
            "Düşük Şiddet (Ha)": float(metrics.get('alan_ha_sinif_2', 0)),
        })
        
if summary_list:
    df_res = pd.DataFrame(summary_list)
    df_res = df_res.sort_values("Yüksek Şiddet (Ha)", ascending=False)
    display(df_res.style.background_gradient(subset=['Yüksek Şiddet (Ha)'], cmap='Reds'))
else:
    print("Sonuç verisi bulunamadı.")

### 5.2. Görsel Sonuçlar (Örneklem)
En çok etkilenen bölgenin detaylı haritaları aşağıdadır.

In [ ]:
# En yüksek hasarlı bölgeyi bul ve göster
if summary_list:
    top_zone = df_res.iloc[0]['Bölge']
    print(f"🔴 En Çok Etkilenen Bölge: {top_zone}")
    
    res = zone_results.get(top_zone)
    if res:
        # dNBR Analizi (Standart Yanmışlık İndeksi)
        if 'dnbr_png' in res:
             display(HTML(f"<h4>{top_zone} - dNBR Analizi</h4>"))
             display(Image(filename=res['dnbr_png'], width=800))


--- 
**Rapor Sonu**
*Bu rapor Python ve Google Earth Engine kullanılarak otomatik olarak üretilmiştir.*